In [77]:
import pandas as pd
import re

In [310]:
xl = pd.ExcelFile('./Tables a299f9a79.xlsx')

In [311]:
xl

In [312]:
dataframes = []
for page in xl.sheet_names:
    df = xl.parse(page, skiprows=14, skip_footer=18)
    df = df.dropna(how='all')
    df = df[df['Rate'] != '(Per 100,000)']
    dataframes.append(df)

In [313]:
maine_surveilance = pd.concat(dataframes).sort_values(by=['Year', 'Location'])

In [314]:
maine_surveilance.Rate = pd.to_numeric(maine_surveilance.Rate, errors='coerce')
maine_surveilance.Year = pd.to_numeric(maine_surveilance.Year, errors='coerce', downcast='integer')

In [315]:
maine_pivot_table = maine_surveilance.pivot_table(values='Rate', columns='Year', index='Location')

In [328]:
just_years = maine_pivot_table[list(range(2008,2018))]
maine_pivot_table['Max_Year'] = just_years.apply(axis=1, func=lambda x: x.idxmax(skipna=True))
maine_pivot_table['Max'] = just_years.apply(axis=1, func=lambda x: x.max(skipna=True))
maine_pivot_table['Mean'] = just_years.apply(axis=1, func=lambda x: x.mean(skipna=True))
#maine_pivot_table['RateofChange'] = (maine_pivot_table[2016].fillna(0)-maine_pivot_table[2008].fillna(0))/maine_pivot_table[2008].fillna(0)

In [329]:
maine_pivot_table.to_csv('./maine_with_stats.csv', na_rep="N/R")

In [330]:
maine_towns = pd.read_csv('./city_names.csv')

In [331]:
statistics = pd.DataFrame(maine_pivot_table.to_records())

location_remap = {
    "Fletchers Landing": "Central Hancock",
    "Prentiss Twp T7 R3 NBPP": "Prentiss",
    "Perkins Twp Swan Island": "Perkins",
    "Monhegan Island": "Monhegan",
    "Hibberts Gore": "Hibberts",
    "Pukakon": "Whitney",
    "Orneville": "Southeast Piscataquis",
    "Pleasant Point": "Passamaquoddy Pleasant Point",
    "Indian Twp Res": "Passamaquoddy Indian Township",
    "T32 MD BPP": "Northwest Hancock"
}

suffix_regex = re.compile(r"\s(Plt|Twp)$")
prefix_regex = re.compile(r"^Saint")

statistics['Location'] = statistics['Location'].str.replace(suffix_regex, '')
statistics['Location'] = statistics['Location'].str.replace(prefix_regex, 'St.')

statistics.replace({'Location': location_remap}, inplace=True)

statistics = statistics.sort_values(by='Location').reset_index()

In [332]:
gis = maine_towns.sort_values(by='NAME10').reset_index()[['OBJECTID', 'NAME10', 'GEOID10', 'COUNTY', 'COUNTYFP10', 'POP10']]

In [333]:
statistics[statistics.Location.str.contains('Indian')]

,index,Location,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,Max_Year,Max,Mean
218,218,Indian Island,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,0.0,0.0
362,219,Passamaquoddy Indian Township,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009.0,0.0,0.0
367,366,Penobscot Indian Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
#suffixes = ['Plt', 'Twp']

In [335]:
locations_dont_exist = []
tgis = gis.loc[-gis.NAME10.isin(locations_dont_exist)].reset_index()

In [336]:
#pd.set_option('display.max_rows', 600)
#pd.concat([statistics, tgis], axis=1)[['Location', 'NAME10']]

In [337]:
joined = statistics.merge(tgis, left_on='Location', right_on='NAME10', how='left')

In [338]:
joined[joined['NAME10'].isnull()]

,index_x,Location,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,...,Max_Year,Max,Mean,index_y,OBJECTID,NAME10,GEOID10,COUNTY,COUNTYFP10,POP10
218,218,Indian Island,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,2008.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
464,459,T3 R1 NBPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
joined.to_csv('./joined_maine_counties.csv', index=False)